In [1]:
! pip install easyocr

In [2]:
! pip install streamlit

In [3]:
! pip install streamlit_option_menu

In [4]:
import streamlit as st
from streamlit_option_menu import option_menu
import easyocr
from PIL import Image
import pandas as pd
import numpy as np
import re
import io
import psycopg2

In [5]:
def image_to_text(path):
    input_img = Image.open(path)

    # converting image to array format
    image_array = np.array(input_img)

    reader = easyocr.Reader(['en'])
    text = reader.readtext(image_array,detail = 0)
    return text,input_img

In [9]:
text_img,input_img = image_to_text("C:/Users/ADMIN/Downloads/1.png")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [ ]:
text_img,input_img = image_to_text("C:/Users/ADMIN/Downloads/2.png")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [ ]:
text_img,input_img = image_to_text("C:/Users/ADMIN/Downloads/3.png")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [ ]:
text_img,input_img = image_to_text("C:/Users/ADMIN/Downloads/4.png")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [ ]:
text_img,input_img = image_to_text("C:/Users/ADMIN/Downloads/5.png")

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [7]:
def extracted_text(texts):
    extrd_dict = {"NAME":[] , "DESIGNATION":[],"COMPANY_NAME": [],"CONTACT":[],"EMAIL":[],"WEBSITE":[],"ADDRESS":[], "PINCODE":[]}

    extrd_dict["NAME"].append(texts[0])
    extrd_dict["DESIGNATION"].append(texts[1])
    
    for i in range(2,len(texts)):
        if texts[i].startswith("+") or (texts[i].replace("-","").isdigit() and '-' in texts[i]):
            extrd_dict["CONTACT"].append(texts[i])
        
        elif "@" in texts[i] and ".com" in texts[i]:
            extrd_dict["EMAIL"].append(texts[i])

        elif "WWW" in texts[i] or "www" in texts[i] or "Www" in texts[i] or "wWw" in texts[i] or "wwW" in texts[i]:
            small = texts[i].lower()
            extrd_dict["WEBSITE"].append(small)

        elif 'Tamil Nadu' in texts[i] or 'TamilNadu' in texts[i] or texts[i].isdigit():
            extrd_dict["PINCODE"].append(texts[i])

        elif re.match(r'^[A-Za-z]',texts[i]):
            extrd_dict["COMPANY_NAME"].append(texts[i])

        else:
            remove_colon = re.sub(r'[,;]','',texts[i])
            extrd_dict["ADDRESS"].append(remove_colon)

    for key,value in extrd_dict.items():
        if len(value)>0:
            concatenate= " ".join(value)
            extrd_dict[key]=[concatenate]

        else:
            value = "NA"
            extrd_dict[key] = [value]

    
    return extrd_dict

In [10]:
text_data = extracted_text(text_img)

In [12]:
df = pd.DataFrame(text_data)
df

,NAME,DESIGNATION,COMPANY_NAME,CONTACT,EMAIL,WEBSITE,ADDRESS,PINCODE
0,Selva,DATA MANAGER,selva digitals,+123-456-7890 +123-456-7891,hello@XYZ1.com,www xyzi.com,123 ABC St Chennai,TamilNadu 600113


In [ ]:
text_img

['SANTHOSH',
 'Technical Manager',
 '123 ABC St , Tirupur, TamilNadu;',
 '641603',
 '+123-456-1234',
 'hello@Sun.com',
 'www.Suncom',
 'Sun Electricals']

In [ ]:
input_img

In [13]:
# converting image to bytes
Image_bytes = io.BytesIO()
input_img.save(Image_bytes,format = "PNG")

image_data = Image_bytes.getvalue()


# Creating dictionary
data = {"IMAGE":[image_data]}

df_1 = pd.DataFrame(data)
concat_df = pd.concat([df,df_1],axis = 1)
concat_df

,NAME,DESIGNATION,COMPANY_NAME,CONTACT,EMAIL,WEBSITE,ADDRESS,PINCODE,IMAGE
0,Selva,DATA MANAGER,selva digitals,+123-456-7890 +123-456-7891,hello@XYZ1.com,www xyzi.com,123 ABC St Chennai,TamilNadu 600113,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...


In [14]:
mydb = psycopg2.connect(host = "localhost",
                        user = "postgres",
                        port= 5432,
                        database = "bizcardx",
                        password = "arjunsoman123"
                           )
cursor = mydb.cursor()

# Table creation
create_table_query = '''create table if not exists bizcard_details(name varchar(225),
                                                                    designation varchar(225),
                                                                    company_name varchar(225),
                                                                    contact varchar(225),
                                                                    email varchar(225),
                                                                    website text,
                                                                    address text,
                                                                    pincode varchar(225),
                                                                    image text)'''
cursor.execute(create_table_query)
mydb.commit()


In [15]:
# Insert query
insert_query = '''INSERT INTO bizcard_details(name,designation,company_name,contact,email,website,address,
                                              pincode,image)
                                              
                                              values(%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
datas = concat_df.values.tolist()[0]
cursor.execute(insert_query,datas)
mydb.commit()

In [16]:
# select query
select_query = "select * from bizcard_details"
cursor.execute(select_query)
table = cursor.fetchall()
mydb.commit()

table_df = pd.DataFrame(table,columns= ("NAME", "DESIGNATION", "COMPANY_NAME", "CONTACT", "EMAIL", "WEBSITE",
                                        "ADDRESS", "PINCODE", "IMAGE"))
table_df

,NAME,DESIGNATION,COMPANY_NAME,CONTACT,EMAIL,WEBSITE,ADDRESS,PINCODE,IMAGE
0,Selva,DATA MANAGER,selva digitals,+123-456-7890 +123-456-7891,hello@XYZ1.com,www xyzi.com,123 ABC St Chennai,TamilNadu 600113,\x89504e470d0a1a0a0000000d49484452000003ec0000...
1,Amit kumar,CEO & FOUNDER,"Erode, GLOBAL INSURANCE St ,",123-456-7569,hello@global.com,www.global.com,123 global TamilNadu,600115,\x89504e470d0a1a0a0000000d49484452000003ec0000...
2,Selva,DATA MANAGER,selva digitals,+123-456-7890 +123-456-7891,hello@XYZ1.com,www xyzi.com,123 ABC St Chennai,TamilNadu 600113,\x89504e470d0a1a0a0000000d49484452000003ec0000...


In [20]:
# select query
select_query = "select NAME from bizcard_details"
cursor.execute(select_query)
table1 = cursor.fetchall()
mydb.commit()

names = []
for i in table1:
    names.append(i[0])

names

['Selva', 'Amit kumar', 'Selva']